## Tutorial: Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2023, 6, 10)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
#!pip install pandas

In [4]:
!pip install langchain
!pip install openai==0.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [5]:
import os
os.environ["OPENAI_API_KEY"] = ""

## LLMChain

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
llm_model

'gpt-3.5-turbo'

Temperature is how creative the model can be.


llm model is gpt 3.5

In [7]:
llm = ChatOpenAI(temperature=0.9,model=llm_model)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
product = "Queen Size Sheet Set"
chain.run(product)

## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Royal Linens Co.
Royal Linens Co. is a leading supplier of high-quality bedding and linens, offering a wide range of products for home and hospitality.

> Finished chain.


'Royal Linens Co. is a leading supplier of high-quality bedding and linens, offering a wide range of products for home and hospitality.'

## SequentialChain

Think about inputs and outputs

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Review"
                    )


In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )


In [ ]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [ ]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [ ]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [ ]:
# review = df.Review[5]
review = "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur... Vieux lot ou contrefaçon !?"
overall_chain(review)



> Entering new SequentialChain chain...



> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur... Vieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold up, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit?!",
 'summary': "The reviewer finds the taste of the product mediocre and suspects it might be an old batch or counterfeit as the foam doesn't hold up and differs from the same product bought in stores.",
 'followup_message': "Réponse: Le critique trouve le goût du produit médiocre et soupçonne qu'il pourrait s'agir d'un lot ancien ou contrefait car la mousse ne tient pas et diffère du même produit acheté en magasin. Il est important de vérifier l'authenticité du produit avant de l'acheter en ligne. Merci pour votre critique honnête."}

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

Main thing that helps the model to understand which template to use

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

RouterOutputParser handles the output type

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

# Assignment

## Task 1: LLM Chain (Large Language Model Chain)

1. Create an LLM Chain that generates creative content based on user prompts. You should generate three different types of content: a story title, a poem, and a joke.
2. For each content type, define a specific prompt that the LLM Chain should use.
3. Run the LLM Chain with each of the three prompts and display the generated content.

langchain and openai libraries are installed.

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import SimpleSequentialChain

Interface with a chat model

In [9]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

Define a chat prompt for each of the three content - joke, story, poem

In [10]:
joke_prompt = ChatPromptTemplate.from_template(
    "Tell me a funny joke about {subject}"
)

In [11]:
story_prompt = ChatPromptTemplate.from_template(
    "Write a story about {subject} not exceeding 500 words and give an appropriate title to the story"
)

In [12]:
poem_prompt = ChatPromptTemplate.from_template(
    "Write a haiku poem about {subject} and give an appropriate title to the story"
)

Define a subject for the content

In [13]:
subject = "Willow Tree"

Use the chain interface of langchain to chain the llm and prompts defined above and maybe an output parser. Use the run method to generate the output

In [14]:
joke_chain = LLMChain(llm=llm, prompt=joke_prompt)

In [15]:
print(joke_chain.run(subject))

Why did the Willow Tree start a band?

Because it wanted to branch out and make some new wood-i-ful music!


In [16]:
story_chain = LLMChain(llm=llm, prompt=story_prompt)

In [17]:
print(story_chain.run(subject))

The Promise of the Willow Tree

In a quiet village nestled between rolling hills, stood a magnificent willow tree. Its branches gracefully reached towards the sky, a vibrant green canopy swaying gently in the breeze. The villagers admired its beauty and sought solace in its shade.

Among the villagers, there lived a young girl named Lily. She had always been captivated by the willow tree's mystic allure. Every evening, she would visit the tree, sitting beneath its comforting branches, sharing her hopes and dreams.

One evening, as Lily sat beneath the willow tree, tears streamed down her cheeks. She longed to explore the world beyond the village but feared leaving her beloved tree behind. The willow tree, sensing her distress, whispered in the wind, "Do not be afraid, dear Lily. Go on your journey, and I promise to be here waiting for your return."

With newfound courage, Lily set off on her adventure, her heart full of excitement and trepidation. She traveled far and wide, experiencin

In [18]:
peom_chain = LLMChain(llm=llm, prompt=poem_prompt, output_parser=StrOutputParser())

In [19]:
print(peom_chain.run(subject))

Whispering branches,
Graceful dancer in the wind,
Ancient tales unfold.

Title: Secrets of the Willow


What if I use the SimpleSequentialChain, the simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.

In [20]:
simple_chain = SimpleSequentialChain(chains=[joke_chain, story_chain, peom_chain],
                                             verbose=True
                                            )

In [21]:
simple_chain.run(subject)



> Entering new SimpleSequentialChain chain...
Why did the willow tree always carry an umbrella?

Because it didn't want to be stuck in a "tree"-mendous downpour!
Title: The Wise Willow and its Umbrella

Once upon a time in a charming little forest, there stood a magnificent willow tree. This particular willow tree was known for its unusual habit of always carrying an umbrella. People often wondered why the willow tree would go through such lengths to carry this accessory wherever it went.

When asked about it, the wise old tree would chuckle softly and say, "Well, my dear friends, I carry this umbrella because I prefer not to be caught in a 'tree'-mendous downpour." The forest creatures found this response rather amusing, but they realized there had to be more to the story.

Curiosity piqued, a group of woodland animals decided to gather around the willow tree one sunny afternoon. A squirrel, a rabbit, and a songbird perched on the willow's sturdy branches, eagerly waiting to unravel

'"The Wise Willow: A Tale of Strength and Preparedness"'

There is more coherence between the content generated. Everything is about the subject with a common theme across joke, story and peom.

## Task 2: Sequential Chain

1. Create two separate LLM Chains: one for generating a product description and another for crafting a social media post.
2. Combine these two chains into a Sequential Chain.
3. Provide a product description prompt and a social media post prompt.
4. Run the Sequential Chain with the prompts and display the generated marketing content.

In [22]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Come up with a quirky product name and description of atleast 200 words describing its uniqueness for the following subject"
    "\n\n{subject}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Product_Description"
                    )

In [23]:
second_prompt = ChatPromptTemplate.from_template(
    "From the point of view of a marketing professional, come up with a Facebook post with emojis and hashtags for the product described below:"
    "\n\n{Product_Description}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="Social_Media_Post"
                    )



In [24]:
from langchain.chains import SequentialChain

In [25]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["subject"],
    output_variables=["Product_Description", "Social_Media_Post"],
    verbose=True
)


In [26]:
overall_chain({"subject": "Water Bottle"})



> Entering new SequentialChain chain...

> Finished chain.


{'subject': 'Water Bottle',
 'Product_Description': "Product Name: AquaJoy\n\nDescription:\n\nIntroducing AquaJoy, the water bottle that will revolutionize your hydration experience! Gone are the days of plain and boring water bottles. Prepare to be delighted by AquaJoy's quirky design and innovative features.\n\nFirst and foremost, AquaJoy is a true fashion statement. With its vibrant and eye-catching colors, this water bottle will turn heads wherever you go. Say goodbye to mundane, transparent bottles and hello to a stylish and quirky accessory that complements your unique personality. Whether you choose the joyful yellow, the bold blue, or the radiant red, AquaJoy will be your ultimate hydration companion.\n\nBut AquaJoy is not only about looks; it also boasts impressive functionality. We understand the importance of carrying the perfect amount of water with you at all times. That's why AquaJoy comes in three different sizes: Small, Medium, and Jumbo. Whether you want a compact bott

## Task 3: Regular Sequential Chain

Choose a text in a non-English language of your choice.

Create three separate chains for the following tasks:

Chain 1: Language translation from the chosen language to English.

Chain 2: Summarization of the translated text (maximum 100 words).

Chain 3: Sentiment analysis of the summarized text.

Combine these three chains into a Regular Sequential Chain.

Run the Regular Sequential Chain with the provided text and display the following:

Translated text.
Text summary.
Sentiment analysis results.

In [27]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to english:"
    "\n\n{Text}"
)
# chain 1: input= Other language Text and output= English Text
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="English_Text"
                    )


In [28]:
# prompt template 2: Summarization of the translated text
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following text in less than 100 words"
    "\n\n{English_Text}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="Summary"
                    )

In [29]:
# prompt template 3: Sentiment Analysis on summarized text
third_prompt = ChatPromptTemplate.from_template(
    "Detect the sentiments in the following summary and list them in one words:\n\n{Summary}. "
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="Sentiments"
                      )

In [30]:
text_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["Text"],
    output_variables=["English_Text", "Summary","Sentiments"],
    verbose=True
)

In [31]:
text = "I nostri lettori hanno potuto osservare che doveva essere trascorso un certo lasso di tempo tra il momento in cui Giove era rientrato dietro gli arazzi e l’istante in cui l’autore della nuova moralità si era così repentinamente svelato all’ammirazione ingenua di Gisquette e Liénarde. Cosa notevole: tutta quella folla, fino a pochi istanti primatanto tumultuosa, ora attendeva con mansuetudine, sulla parola del commediante; il che prova la seguente verità eterna e ogni giorno verificabile nei nostri teatri, che il modo migliore per far aspettare pazientemente il pubblico è dirgli che si sta per cominciare da un momento all’altro. Tuttavia Joannes lo studente non dormiva. – Ehilà! gridò d’un tratto nel bel mezzo della placida attesa che era subentrata al trambusto, Giove, madonna Vergine, saltimbanchi della malora! ci prendete in giro? lo spettacolo! lo spettacolo! Cominciate, o ricominciamo! Non ci volle altro. Una musica di strumenti alti e bassi si fece udire dall’interno dell’impalcatura; gli arazzi si sollevarono; ne uscirono quattro personaggi variopinti e imbellettati, si arrampicarono su per la ripida scala del teatro e, giunti sulla piattaforma superiore, si allinearono in bell’ordine davanti al pubblico, che salutarono con un profondo inchino; allora la sinfonia tacque. Era l’inizio del mistero. "
text_chain(text)



> Entering new SequentialChain chain...

> Finished chain.


{'Text': 'I nostri lettori hanno potuto osservare che doveva essere trascorso un certo lasso di tempo tra il momento in cui Giove era rientrato dietro gli arazzi e l’istante in cui l’autore della nuova moralità si era così repentinamente svelato all’ammirazione ingenua di Gisquette e Liénarde. Cosa notevole: tutta quella folla, fino a pochi istanti primatanto tumultuosa, ora attendeva con mansuetudine, sulla parola del commediante; il che prova la seguente verità eterna e ogni giorno verificabile nei nostri teatri, che il modo migliore per far aspettare pazientemente il pubblico è dirgli che si sta per cominciare da un momento all’altro. Tuttavia Joannes lo studente non dormiva. – Ehilà! gridò d’un tratto nel bel mezzo della placida attesa che era subentrata al trambusto, Giove, madonna Vergine, saltimbanchi della malora! ci prendete in giro? lo spettacolo! lo spettacolo! Cominciate, o ricominciamo! Non ci volle altro. Una musica di strumenti alti e bassi si fece udire dall’interno del

## Task 4: Router Chain (Combining All Concepts)

Use case: Customer support for a travel booking platform

Imagine you are developing a customer support system for a travel booking platform. The platform provides services related to **flight bookings, hotel reservations, and car rentals**. Your task is to create a Router Chain that routes **customer inquiries** to the appropriate sub chains based on the nature of the inquiry.

Instructions:

Create three separate LLM Chains, each specialized for handling specific types of customer inquiries related to the travel booking platform: Flight Booking, Hotel Reservation, and Car Rental. Define prompts for each category.

Create a Router Chain that routes customer inquiries to the appropriate subchain based on the content of the inquiry. The Router Chain should have rules to identify keywords or phrases that indicate the inquiry category.

Provide a **list of sample customer inquiries **related to flight bookings, hotel reservations, and car rentals. Include inquiries that cover a range of scenarios and issues.


Run the Router Chain with the sample inquiries and display the responses for each category. Ensure that inquiries are routed correctly to the relevant subchains.

***Templates***

In [32]:
flight_template = """You're the sole point of contact for the Flight Booking arm of a reputed travel booking platform. \
You are great at answering customer enquiries regarding \
how to book flights on the platform, flight availability and all things related to flight schedules and departures\
in a concise and easy to understand manner. You can make up the information as you're responsible for everything regarding flights for the \
travel booking platform.  \
When you don't know the answer to a question you admit that you don't know and politely ask them to call/email customer service.\
Remember, Customer is King.\
Here is a customer enquiry:
{input}"""


hotel_template = """You the sole point of contact for the Hotel Reservation arm of a reputed travel booking platform.. \
You are great at answering customer enquiries regarding \
how to make reservations for hotels on the platform, room availability and all things related to check-ins and check-outs\
in a concise and easy to understand manner. You can make up the information as you're responsible for everything regarding hotels for the \
travel booking platform.  \
When you don't know the answer to a question you admit that you don't know and politely ask them to call/email customer service.\
Remember, Customer is King.\
Here is a  customer enquiry:
{input}"""

car_template = """You the sole point of contact for the Car Rental arm of a reputed travel booking platform.. \
You are great at answering customer enquiries regarding \
how to make reservations for rental cars on the platform, requirements for making reservations, \
car availability and all things related to cars - model, miles ran after service. \
in a concise and easy to understand manner. You can make up the information as you're responsible for everything regarding car rentals for the \
travel booking platform.  \
When you don't know the answer to a question you admit that you don't know and politely ask them to call/email customer service.\
Make sure you answer questions only regarding cars rentals on your platform. \
Remember, Customer is King.\
Here is a  customer enquiry:
{input}"""



In [33]:
prompt_infos = [
    {
        "name": "flights",
        "description": "Good for answering enquiries about flights",
        "prompt_template": flight_template
    },
    {
        "name": "hotels",
        "description": "Good for answering enquiries about hotels",
        "prompt_template": hotel_template
    },
    {
        "name": "cars",
        "description": "Good for answering questions about car rentals",
        "prompt_template": car_template
    }
]

In [34]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [35]:
llm = ChatOpenAI(temperature=0.8, model=llm_model)

In [36]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [37]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [38]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [39]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [40]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

### Sample Flight enquiries

In [41]:
chain.run("What are the flights available and their schedule between San Fransisco and Seattle on 29th January")



> Entering new MultiPromptChain chain...
flights: {'input': 'What are the flights available and their schedule between San Fransisco and Seattle on 29th January'}
> Finished chain.


"Thank you for reaching out to us with your query regarding flights between San Francisco and Seattle on 29th January. I will be happy to assist you with the flight availability and schedule.\n\nCurrently, we have multiple flights available between San Francisco and Seattle on 29th January. Here are some options for you:\n\n1. Flight XYZ123:\n   - Departure: San Francisco at 9:00 AM\n   - Arrival: Seattle at 11:00 AM\n   - Airline: ABC Airlines\n\n2. Flight DEF456:\n   - Departure: San Francisco at 12:30 PM\n   - Arrival: Seattle at 2:30 PM\n   - Airline: XYZ Airways\n\n3. Flight GHI789:\n   - Departure: San Francisco at 3:15 PM\n   - Arrival: Seattle at 5:15 PM\n   - Airline: QRS Airlines\n\nPlease note that these flight schedules are subject to change and availability. It is always recommended to double-check the flight timings before making your final booking.\n\nIn case you require more specific flight options or have any other questions, I apologize that I don't have access to rea

In [42]:
chain.run("I won't be able to make it for my flight scheduled in 30 minutes. What can I do? Can you help me book another flight?")



> Entering new MultiPromptChain chain...
flights: {'input': "I won't be able to make it for my flight scheduled in 30 minutes. What can I do? Can you help me book another flight?"}
> Finished chain.


"I'm sorry to hear that you won't be able to make it for your flight scheduled in 30 minutes. As the sole point of contact for flight bookings, I'm here to assist you with your queries. However, please note that I cannot make new flight bookings directly. \n\nIn this situation, I recommend contacting our customer service team immediately. They have the expertise and access to the necessary tools to assist you in booking another flight. You can reach them by calling our dedicated customer service hotline or by sending an email to our customer support email address. They will be able to guide you through the process and help you find the best available options based on your specific needs.\n\nRemember, our customer service team is dedicated to providing you with the best possible assistance, so don't hesitate to reach out to them for any flight-related queries or concerns."

In [43]:
chain.run("Can you give me detailed instructions on how to book flights on your platform?")



> Entering new MultiPromptChain chain...
flights: {'input': 'Can you give me detailed instructions on how to book flights on your platform?'}
> Finished chain.


'Of course! Booking flights on our platform is quick and convenient. Here are detailed instructions on how to book flights:\n\n1. Visit our website or open our mobile app.\n\n2. On the homepage, you will find a search bar. Enter your desired departure and destination airports, along with your preferred travel dates.\n\n3. Click on the "Search" button to initiate the flight search.\n\n4. A list of available flights will be displayed, along with important details such as airlines, flight times, and prices. You can also use filters to narrow down your search based on preferences like price, number of stops, or specific airlines.\n\n5. Once you have found the perfect flight for your needs, select it by clicking on the "Select" button.\n\n6. Review the flight details, including the fare breakdown and any additional charges or baggage policies. Make sure to carefully read and understand all the terms and conditions.\n\n7. If you are satisfied with the selected flight, proceed to the next ste

### Sample Hotel Reservation enquiries

In [44]:
chain.run("Can you give me detailed instructions on how to book hotels on your platform?")



> Entering new MultiPromptChain chain...
hotels: {'input': 'Can you give me detailed instructions on how to book hotels on your platform?'}
> Finished chain.


'Thank you for reaching out to us for assistance in booking hotels on our platform. I\'d be happy to guide you through the process. \n\nTo book hotels on our platform, please follow these simple steps:\n\n1. Start by visiting our website or mobile app and enter your destination in the search bar. You can also specify your check-in and check-out dates, as well as the number of rooms and guests.\n\n2. Once you\'ve entered the necessary details, click on the search button to view a list of available hotels in your chosen destination.\n\n3. You can then filter the search results based on your preferences, such as price range, star rating, amenities, and more. This will help you narrow down the options and find the perfect hotel for your needs.\n\n4. Once you\'ve found a hotel that suits your requirements, click on the "Book Now" or "Select Room" button. This will take you to a page where you can choose your desired room type and view the total price for your stay.\n\n5. After selecting the

In [45]:
chain.run("Do I have the Early Check-in option for my hotel?")



> Entering new MultiPromptChain chain...
hotels: {'input': 'Do I have the Early Check-in option for my hotel?'}
> Finished chain.


"Thank you for reaching out to us! We appreciate your interest in the early check-in option for your hotel reservation. \n\nAt our travel booking platform, we understand the importance of a comfortable and convenient stay for our customers. While we strive to provide early check-in options whenever possible, it ultimately depends on the specific hotel's policies and availability.\n\nTo determine if early check-in is available for your reservation, we recommend contacting our customer service team directly. They have access to the most up-to-date information regarding hotel policies and can assist you in checking the availability of early check-in for your specific hotel.\n\nYou can reach our customer service team either by calling our dedicated helpline or by sending an email. They will be more than happy to assist you with your inquiry and provide you with the necessary information.\n\nWe apologize for any inconvenience this may cause and appreciate your understanding. Should you have

In [46]:
chain.run("I am travelling to Orlando. What are the budget hotels available near the airport and their room rates?")



> Entering new MultiPromptChain chain...
hotels: {'input': 'I am travelling to Orlando. What are the budget hotels available near the airport and their room rates?'}
> Finished chain.


"Thank you for choosing our travel booking platform for your trip to Orlando! I'll be happy to assist you with finding budget hotels near the airport.\n\nWe have several budget-friendly options available near Orlando airport. Here are a few to consider along with their approximate room rates:\n\n1. Best Budget Inn Orlando: This hotel offers rooms starting at $50 per night.\n\n2. Days Inn by Wyndham Orlando Airport Florida Mall: Room rates at this hotel start from $60 per night.\n\n3. Red Roof Inn Orlando South - Florida Mall: You can find rooms at this hotel starting from $70 per night.\n\n4. Quality Inn & Suites Orlando Airport: Room rates at this hotel begin at $80 per night.\n\nPlease note that these rates are approximate and subject to change based on availability and seasonal variations. To get the most accurate and up-to-date information about room availability and rates, I recommend contacting our customer service team at [customer service number] or via email at [customer servi

### Sample Car Rental Enquiries

In [47]:
chain.run("I am travelling to Chicago. Do you have cars for rent available to pickup at the airport? ")



> Entering new MultiPromptChain chain...
cars: {'input': 'I am travelling to Chicago. Do you have cars for rent available to pickup at the airport?'}
> Finished chain.


'Yes, we do have cars available for rent at the airport in Chicago. Our platform offers a wide range of rental car options that you can choose from. To check the availability and make a reservation, you can visit our website or use our mobile app. Simply enter your travel details, including the dates and location of pickup at the Chicago airport, and our system will display the available car options for you to choose from. If you have any specific requirements or preferences, such as car model or mileage, you can filter your search accordingly. We strive to provide a seamless and convenient car rental experience for our customers.'

In [48]:
chain.run("Can I get an SUV to drive around Page in Arizona")



> Entering new MultiPromptChain chain...
cars: {'input': 'Can I rent an SUV to drive around Page in Arizona'}
> Finished chain.


"Absolutely! We offer a wide range of SUVs for rent on our platform, including options suitable for driving around Page, Arizona. To make a reservation, simply visit our website or mobile app, select Page as your pickup location, and choose the SUV category. You can then browse through the available SUV models and select the one that best suits your needs. \n\nAs for the requirements, you will need a valid driver's license and a credit card in the name of the primary driver at the time of pickup. Additionally, please note that the minimum age to rent a car is typically 21 years, although some car rental companies may require renters to be at least 25 years old. \n\nRegarding car availability, we strive to ensure a wide variety of vehicles are available for rent in each location, including Page, Arizona. However, availability may vary depending on the dates of your rental and the demand in that particular area. We recommend booking in advance to secure your preferred SUV model.\n\nIn te

In [49]:
chain.run("My rental car has broke down. What can I do?")



> Entering new MultiPromptChain chain...
cars: {'input': 'My rental car has broke down. What can I do?'}
> Finished chain.


"I'm sorry to hear about the situation with your rental car. As the point of contact for car rentals on our platform, I'll do my best to assist you.\n\nIf your rental car has broken down, the first step is to contact the car rental company directly. They will be able to provide immediate assistance and guidance on what to do next. They may arrange for a tow truck, offer a replacement vehicle, or provide instructions on how to proceed.\n\nIt's important to remember that each car rental company has its own policies and procedures for handling breakdowns. They are responsible for providing support and resolving any issues related to the rental car.\n\nIf you're unable to reach the car rental company or need further assistance, I recommend contacting our customer service team. They can provide additional guidance and help facilitate communication with the car rental company.\n\nPlease let me know if there's anything else I can assist you with or if you have any further questions."

### An unrelated customer

In [50]:
chain.run("I am hungry.")



> Entering new MultiPromptChain chain...
None: {'input': 'I am hungry.'}
> Finished chain.


"I'm sorry to hear that. Can I help you find a recipe or suggest a nearby restaurant?"

## Task 5: Custom Chain

If you wish to demonstrate your creativity and explore a different NLP task:

Choose a specific NLP task you find interesting (e.g., language translation, summarization, sentiment analysis, image captioning, etc.).


Create a custom chain for this task, outlining the steps involved in your chain.


Provide examples of input data and expected output.
Run your custom chain with sample data and showcase the results.



For Task 5 (Custom Chain), provide a brief description of the chosen task and why you find it interesting.

### Failed Attempt

Image Captioning agents and further processing

In [ ]:
!pip install transformers
!pip install pillow

In [ ]:
from google.colab import files
from transformers import BlipProcessor, BlipForConditionalGeneration, DetrImageProcessor, DetrForObjectDetection
from PIL import Image

In [ ]:
# Ask the user to upload an image file
uploaded = files.upload()


In [ ]:
# Check if any files were uploaded
if uploaded:
    # Get the first uploaded file
    img_path = next(iter(uploaded))

    # Open the uploaded image and convert it to RGB
    image = Image.open(img_path).convert('RGB')

    # Define the model and device
    model_name = "Salesforce/blip-image-captioning-large"
    device = "cpu"  # You can change this to "cuda" if you have a GPU available

    # Initialize the Blip processor and model
    processor = BlipProcessor.from_pretrained(model_name)
    model = BlipForConditionalGeneration.from_pretrained(model_name).to(device)

    # Process the image and generate a caption
    inputs = processor(image, return_tensors='pt').to(device)
    output = model.generate(**inputs, max_new_tokens=20)

    # Decode and print the generated caption
    caption = processor.decode(output[0], skip_special_tokens=True)
    print("Generated Caption:", caption)
else:
    print('No files were uploaded.')

###Correct Attempt
Final Answer


Scenario: Multilingual Movie Release


More and more movies are being produced and released worldwide.
Often times, translation, subtitling and making the audio available in multiple languages is a strenous and manual task.
NLP and generative models can potentially revolutionize this field, making accessible the magic of movies to all motion picture lovers worldwide.
Here is a very small attempt at coming up with promotional content for a movie in English language in different languages. This is a very basic attempt. It can be thought of as a first step to the greater vision described above.





Chain 1: Given a subject, come up with the synopsis of a movie

Chain 2: Given the synopsis, come up with a title for the movie

Chain 3: Given the synopsis and title, create a social media post announcing the movie release

Chain 4: Given the title and synopsis, write a movie review

Chain 5: Given the language, come up with a foreign language title of the movie

Chain 6: Translate the social media post from chain 3 to the language specified

Run the Regular Sequential Chain with the provided text and display the following:

Synopsis. Title. Social Media Post. Review. Foreign Title. Foreign Language Social Media Post

In [26]:
from langchain.chains import SequentialChain

In [27]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: synopsis of a movie
first_prompt = ChatPromptTemplate.from_template(
    "Given the subject, come up with the synopsis for a movie"
    "\n\n{Subject}"
)
# chain 1: input= Subject and output= Synopsis
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Synopsis"
                    )


In [28]:
# prompt template 2: title for the movie
second_prompt = ChatPromptTemplate.from_template(
    "Can you give a catchy title for a movie with the following synopsis:"
    "\n\n{Synopsis}"
)
# chain 2: input= Synopsis and output= Title
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="Title"
                    )


In [29]:
# prompt template 3: Social Media Post
third_prompt = ChatPromptTemplate.from_template(
    "Given the title and synopsis of the movie, can you come up with a social media post\
    announcing it's release the coming Friday.\
    \n\nTitle: {Title}\n\n Synopsis: {Synopsis}"
)
# chain 3: input= Title, Synopsis and output= Social Media Post
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="Social Media Post"
                      )


In [30]:
# prompt template 4: Review
fourth_prompt = ChatPromptTemplate.from_template("Think like a movie critic and for the title and synopsis of a movie\
given below, come up with a movie review for the New York Times \n\nTitle: {Title}\n\n Synopsis: {Synopsis}")
# chain 4: input= Title, Synopsis and output= Review
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="Review"
                     )

In [31]:
# prompt template 5: Foreign Title
fifth_prompt = ChatPromptTemplate.from_template("To release the movie in non-English speaking countries, we need to have an alternate title\
in their respective languages. Given the title, synopsis of a movie, come up with a foreign title for the movie in th language specified below.\
  \n\nTitle: {Title}\n\n Synopsis: {Synopsis}\n\n Language:{Language}")
# chain 5: input= Title, Synopsis and output= Foreign Title
chain_five = LLMChain(llm=llm, prompt=fifth_prompt,
                      output_key="Foreign Title"
                     )

In [32]:
# prompt template 6: Translated Social Media Post
sixth_prompt = ChatPromptTemplate.from_template("Translate the following social media post about the movie to the language specified below. \
Replace the title of the movie with the Foreign Title given.\n\n Social Media Post: {Social Media Post} Foreign Title of the Movie: {Foreign Title}\
\n\n Language:{Language}")
# chain 6: input= Social Media Post and output= Translated Social Media Post
chain_six = LLMChain(llm=llm, prompt=sixth_prompt,
                      output_key="Translated Social Media Post"
                     )

In [33]:
# overall_chain: input= Subject, Language
# and output= Synopsis, Title,Social Media Post, Review, Foreign Title, Translated Social Media Post
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four, chain_five, chain_six],
    input_variables=["Subject", "Language"],
    output_variables=["Synopsis", "Title","Social Media Post", "Review", "Foreign Title", "Translated Social Media Post"],
    verbose=True
)

In [34]:
subject = "Love in times of Corona"
language = "French"
overall_chain({'Subject': subject,'Language': language})



> Entering new SequentialChain chain...

> Finished chain.


{'Subject': 'Love in times of Corona',
 'Language': 'French',
 'Synopsis': '"Love in times of Corona" follows the intertwining journeys of four individuals navigating the complexities of romance during a global pandemic. Mark, a successful businessman, finds himself unexpectedly single when his long-distance relationship crumbles due to the strain of lockdowns. Meanwhile, Sarah, a married woman, struggles with the challenges of maintaining a spark in her relationship while confined to the same four walls. These two cross paths virtually, igniting a profound connection that challenges their moral boundaries. Simultaneously, John, a frontline healthcare worker, meets Maria, an artist who copes with isolation through her creative expression. As they confront the fear and uncertainty that the pandemic brings, John and Maria develop an unexpected bond that teaches them the true meaning of love and self-sacrifice. "Love in times of Corona" explores the resilience of the human spirit, remindi

## Task 6: Final Task

For all the tasks 1-4. You have been given a scenario and asked to implement the Lang Chain. Now, think of another scenario where you can use the chains and implement them.



Instructions:



Design a scenario-specific task or problem statement related to your chosen scenario. Clearly define the objectives and expected outcomes.


Provide clear documentation and comments within your code to explain the scenario, the purpose of each chain, and how they work together to solve the problem.


Present the results, outputs, or responses generated by your LangChains based on your chosen scenario.

### LLM Chain task 1
Given a technology related topic/area, come up with a company name

In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [37]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [38]:
brand_prompt = ChatPromptTemplate.from_template(
    "Given a technology related topic/area - {topic}, come up with a witty brand name for a startup company to be established in that space."
)

In [46]:
topic = "Observability and Monitoring for Software Applications"

In [47]:
brand_chain = LLMChain(llm=llm, prompt=brand_prompt)
print(brand_chain.run(topic))

TechSight


### Simple Sequential Chain task 2
Given a subject, come up with a poem or prose written in Iambic pentameter.
Give an appropriate title in Shakespearean English to the poem

In [50]:
from langchain.chains import SimpleSequentialChain

In [54]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Given a subject, come up with a poem or prose written in Iambic pentameter.\n\n Subject: {subject} "
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Text"
                    )
second_prompt = ChatPromptTemplate.from_template(
    "What would William Shakespeare give as title to the following text. Please ensure the title is in Shakespearean English\n\n Text: {Text}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="Title"
                    )

In [56]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [57]:
subject = "Sunflower"

In [58]:
overall_simple_chain.run(subject)



> Entering new SimpleSequentialChain chain...
In fields of gold, a sunflower stands tall and proud,
Its vibrant petals, a beacon in the golden shroud.
With face uplifted, gazing towards the azure skies,
It basks in warmth, its spirit reaching new highs.

Each petal, like a ray of sunlight, gleams so bright,
A vibrant palette, a masterpiece of pure delight.
The sunflower's beauty, a marvel to behold,
A testament to nature's wonders, a tale untold.

With roots so deep, firmly anchored in the earth,
It stands resilient, bestowing joy and mirth.
Its slender stalk, a testament to strength and grace,
Nature's gentle dance, it elegantly embraces.

As the sun sets, the sunflower seeks its last ray,
A final salute before the night overtakes the day.
Yet in its heart, a promise of a brand new dawn,
A celebration of life, an eternal bond.

Oh, sunflower, you symbolize the sun's embrace,
A symbol of resilience, hope, and inner grace.
In your majestic presence, beauty truly shines,
A reminder of 

'"The Sunflower\'s Radiant Grace"'

### Sequential Chain task 3
Given a subject come up with a 500 word essay.

Given the essay, summarize it.


Translate the summarized text to a language specified.

Run a Regular Sequential Chain combining all the three tasks and output the results


In [59]:
# prompt template 1: Essay
first_prompt = ChatPromptTemplate.from_template(
    "Come up with a 500 word essay on the following topic"
    "\n\nTopic: {Topic}"
)
# chain 1: input= Topic and output= Essay
chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="Essay"
                    )


In [60]:
# prompt template 2: Summarization of the essay
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following text in less than 100 words"
    "\n\n{Essay}"
)
# chain 2: input= Essay and output= Summary
chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="Summary"
                    )

In [61]:
# prompt template 3: Translation of Summary
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to the language specified.\n\n\
     Text: {Summary}\n\
     Language: {Language}"
)
# chain 3: input= Summary, Language and output= Translated_Summary
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="Translated_Summary"
                      )

In [62]:
text_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["Topic","Language"],
    output_variables=["Essay", "Summary","Translated_Summary"],
    verbose=True
)

In [63]:
topic = "Children: The Wealth of a Nation"
language = "German"

In [65]:
text_chain({'Topic': topic,'Language': language})



> Entering new SequentialChain chain...

> Finished chain.


{'Topic': 'Children: The Wealth of a Nation',
 'Language': 'German',
 'Essay': "Children: The Wealth of a Nation\n\nThe future of a nation lies in the hands of its children. They are the foundation upon which progress is built, and their well-being is crucial to the overall prosperity of a country. Children hold immense potential, and nurturing their growth and development is essential for the success of any nation. In this essay, we will explore why children are considered the wealth of a nation and discuss the various factors that contribute to their well-being.\n\nOne of the primary reasons children are seen as the wealth of a nation is due to their role as future leaders. They are the ones who will shape and determine the direction of a country in the coming years. Investing in their education and providing them with ample opportunities to learn and grow is paramount. A well-educated and skilled child population will lead to a more prosperous and competitive society in the global a

### Router chain task 4

Usecase: Basic AI Pair Programmer



We are developing a simple AI Pair Programmer which could help with syntax and general questions for the following programming languages


1. Java

2. C++

3. Python



The system helps with doubts related to different programming languages. The task is to create a Router Chain that routes questions to the appropriate sub chains based on the language.

Steps involved:

Create three separate LLM Chains, each specialized for handling specific types questions related to the programming languages supported: Java, C++ and Python. Define prompts for each category.

Create a Router Chain that routes developer queries to the appropriate subchain based on the content of the query. The Router Chain should have rules to identify keywords or phrases that indicate the field of interest.

Provide a list of sample queries.

Run the Router Chain with the sample queries and display the responses for each category. Ensure that questions/doubts are routed correctly to the relevant subchains.

In [67]:
java_template = """You are very knowledgable in Java programming language. \
You are great at answering questions about Java in a concise\
and easy to understand manner with code examples. \
When you don't know the answer to a question you admit\
that you don't know. Ensure that the answers are based on Java version 1.8. \

Here is a question:
{input}"""


cpp_template = """You are knowledgable in C++ programming language. \
You are great at answering C++ questions about syntax, standard C++ template library as well as general enquiries on C++. \
with code examples.\

Here is a question:
{input}"""

python_template = """You are knowledgable inPython programming language. \
You are great at answering C++ questions about syntax and standard python libraries \
with code examples.\

Here is a question:
{input}"""


In [68]:
prompt_infos = [
    {
        "name": "java",
        "description": "Good for answering questions about Java",
        "prompt_template": java_template
    },
    {
        "name": "cpp",
        "description": "Good for answering C++ questions",
        "prompt_template": cpp_template
    },
    {
        "name": "python",
        "description": "Good for answering Python questions",
        "prompt_template": python_template
    }
]

In [69]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [70]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [71]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [72]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [73]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [74]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [75]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [78]:
print(chain.run("What is the Collection used to implement Heaps in Java"))



> Entering new MultiPromptChain chain...
java: {'input': 'What is the Collection used to implement Heaps in Java'}
> Finished chain.
The Collection used to implement Heaps in Java is the PriorityQueue class. PriorityQueue is a priority queue implementation based on a priority heap. It allows elements to be inserted in any order and provides efficient retrieval of the highest-priority element. Here's an example of how to use PriorityQueue to implement a heap:

```java
import java.util.PriorityQueue;

public class HeapExample {
    public static void main(String[] args) {
        // Create a PriorityQueue
        PriorityQueue<Integer> heap = new PriorityQueue<>();

        // Insert elements into the heap
        heap.add(5);
        heap.add(2);
        heap.add(8);
        heap.add(1);
        heap.add(10);

        // Retrieve and remove the highest-priority element
        int highestPriority = heap.poll();
        System.out.println("Highest priority element: " + highestPriority)

In [80]:
print(chain.run("How do you declare 2 dimensional vectors in C++?"))



> Entering new MultiPromptChain chain...
cpp: {'input': 'How do you declare 2 dimensional vectors in C++?'}
> Finished chain.
To declare a 2-dimensional vector in C++, you can use the `vector` container from the Standard Template Library (STL). Here's an example:

```cpp
#include <iostream>
#include <vector>

int main() {
    // Declare a 2-dimensional vector
    std::vector<std::vector<int>> matrix;

    // Add elements to the vector
    matrix.push_back({1, 2, 3});
    matrix.push_back({4, 5, 6});
    matrix.push_back({7, 8, 9});

    // Access and print elements of the vector
    for (const auto& row : matrix) {
        for (const auto& element : row) {
            std::cout << element << " ";
        }
        std::cout << std::endl;
    }

    return 0;
}
```

In this example, we declare a 2-dimensional vector `matrix` using `std::vector<std::vector<int>>`. Each element of the outer vector represents a row, and each element of the inner vector represents an element in that row.


In [82]:
print(chain.run("How can measure time taken for an operation in Python?"))



> Entering new MultiPromptChain chain...
python: {'input': 'How can I measure the time taken for an operation in Python?'}
> Finished chain.
In Python, you can measure the time taken for an operation using the `time` module. There are a few different approaches you can take depending on your specific requirements. Here are a couple of examples:

1. Using `time.time()`:
```python
import time

start_time = time.time()

# Perform your operation here
# ...

end_time = time.time()
execution_time = end_time - start_time

print("Time taken:", execution_time, "seconds")
```

In this approach, you capture the start time using `time.time()` before the operation and the end time after the operation. The difference between the two gives you the execution time in seconds.

2. Using `timeit` module:
```python
import timeit

# Define the operation as a function
def my_operation():
    # Perform your operation here
    # ...

# Measure the execution time using timeit
execution_time = timeit.timeit(m

Testing default chain

In [83]:
print(chain.run("Is Javascript a strongly typed language?"))



> Entering new MultiPromptChain chain...
None: {'input': 'Is Javascript a strongly typed language?'}
> Finished chain.
No, JavaScript is not a strongly typed language. It is a dynamically typed language, which means that variables can hold values of any type and their types can be changed at runtime.
